In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import csv
import seaborn as sns
import ipywidgets as widgets
import DLC_sqOF_v1 as DLCsq
import DLC_cyOF_v1 as DLCcy
import DLC_plot_v1 as DLCp
import ast
from pathlib import Path
from functools import reduce
from itertools import *
from scipy import stats

In [ ]:
datapath = '/Users/Documents/...'

In [ ]:
save_dir = os.path.join(datapath + "/Figures")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
genotype_options = ['Choose', 'MitoPark', 'DAT', 'Anxa1']; genotype = 'None'
manipulation_options = ['Choose', '', 'Casp3', 'TeLC']; manipulation = 'None'

drop_down1 = widgets.Dropdown(options = genotype_options, value='Choose', description='genotype:')
drop_down2 = widgets.Dropdown(options = manipulation_options, value='Choose', description='manipulation:')

def dropdown_handler_genotype(change):
    global genotype
    print(change.new); genotype = change.new 

def dropdown_handler_manipulation(change):
    global manipulation
    print(change.new); manipulation = change.new 
   
drop_down1.observe(dropdown_handler_genotype, names='value')
drop_down2.observe(dropdown_handler_manipulation, names='value')
display(drop_down1, drop_down2)

In [ ]:
day = 'D1'
midbrain = 'SNc'
duration = '5_20min'

if genotype == 'MitoPark':
    data_ctrl1,data_test1 = DLCp.processed_data(genotype, '', 'SquareOF', '8wks', day, datapath, midbrain, duration)
    data_ctrl2,data_test2 = DLCp.processed_data(genotype, '', 'SquareOF', '16wks', day, datapath, midbrain, duration)
    data_ctrl3,data_test3 = DLCp.processed_data(genotype, '', 'SquareOF', '24wks', day, datapath, midbrain, duration)
    data_ctrl4,data_test4 = DLCp.processed_data(genotype, '', 'Cylinder', '8wks', day, datapath, midbrain, '')
    data_ctrl5,data_test5 = DLCp.processed_data(genotype, '', 'Cylinder', '16wks', day, datapath, midbrain, '')
    data_ctrl6,data_test6 = DLCp.processed_data(genotype, '', 'Cylinder', '24wks', day, datapath, midbrain, '')
    
    animal = [data_ctrl1,data_test1,data_ctrl2,data_test2,data_ctrl3,data_test3,
              data_ctrl4,data_test4,data_ctrl5,data_test5,data_ctrl6,data_test6]
    df_combined_sq = pd.concat([animal[0],animal[1],animal[2],animal[3],animal[4],animal[5]])
    df_combined_cy = pd.concat([animal[6],animal[7],animal[8],animal[9],animal[10],animal[11]])
    dataframe = [df_combined_sq, df_combined_cy]

elif genotype == 'DAT':
    data_ctrl1,data_test1 = DLCp.processed_data(genotype, manipulation, 'SquareOF', '', day, datapath, midbrain, duration)
    data_ctrl2,data_test2 = DLCp.processed_data(genotype, manipulation, 'Cylinder', '', day, datapath, midbrain, '')
    animal = [data_ctrl1,data_test1,data_ctrl2,data_test2]
    df_combined_sq = pd.concat([animal[0], animal[1]])
    df_combined_cy = pd.concat([animal[2], animal[3]])
    dataframe = [df_combined_sq, df_combined_cy]

elif genotype == 'Anxa1':
    data_ctrl1,data_test1 = DLCp.processed_data(genotype, manipulation, 'SquareOF', '', day, datapath, midbrain, duration)
    data_ctrl2,data_test2 = DLCp.processed_data(genotype, manipulation, 'Cylinder', '', day, datapath, midbrain, '')
    animal = [data_ctrl1,data_test1,data_ctrl2,data_test2]
    df_combined_sq = pd.concat([animal[0], animal[1]])
    df_combined_cy = pd.concat([animal[2], animal[3]])
    dataframe = [df_combined_sq, df_combined_cy]

In [ ]:
# convert string list to python arrays for further analysis
df_combined_sq['stop_frames_len_list'] = df_combined_sq['stop_frames_len_list'].apply(ast.literal_eval)
df_combined_sq['move_frames_len_list'] = df_combined_sq['move_frames_len_list'].apply(ast.literal_eval)

# set stop_th and move_th here for range in figures below
stop_th = df_combined_sq['stop_th(cm/s)'].iloc[0] 
move_th = df_combined_sq['move_th(cm/s)'].iloc[0] 

In [ ]:
xlim_min = 0
xlim_max = 3
ylim_min = 0
ylim_max = 6

DLCp.DLC_histgram(genotype, manipulation, dataframe, save_dir, 'stop', stop_th, xlim_min, xlim_max, day, midbrain, duration)
DLCp.DLC_histgram(genotype, manipulation, dataframe, save_dir, 'move', move_th, xlim_min, xlim_max, day, midbrain, duration)

if genotype == 'MitoPark':
    DLCp.DLC_histgram_MP_age(genotype, dataframe, save_dir,'stop', stop_th, xlim_min, xlim_max, ylim_min, ylim_max, day, midbrain, duration)
    DLCp.DLC_histgram_MP_age(genotype, dataframe, save_dir,'move', move_th, xlim_min, xlim_max, ylim_min, ylim_max, day, midbrain, duration)

In [ ]:
# convert string list to python arrays for further analysis
df_combined_sq['immobile_speed_list'] = df_combined_sq['immobile_speed_list'].apply(ast.literal_eval)
df_combined_sq['mobile_speed_list'] = df_combined_sq['mobile_speed_list'].apply(ast.literal_eval)

In [ ]:
xlim_min_i = 0
xlim_max_i = 0.5
ylim_min_i = 0
ylim_max_i = 10

xlim_min_m = 0
xlim_max_m = 30
ylim_min_m = 0
ylim_max_m = 0.2

DLCp.DLC_histgram_speed(genotype, manipulation, dataframe, save_dir, 'immobile', stop_th, xlim_min_i, xlim_max_i, ylim_min_i, ylim_max_i, day, midbrain, duration, bins_speed = np.arange(0, 30, step=0.1))
DLCp.DLC_histgram_speed(genotype, manipulation, dataframe, save_dir, 'mobile', move_th, xlim_min_m, xlim_max_m, ylim_min_m, ylim_max_m, day, midbrain, duration, bins_speed = np.arange(0, 30, step=1))

if genotype == 'MitoPark':
    DLCp.DLC_histgram_MP_age_speed(genotype,dataframe, save_dir,'immobile', stop_th, xlim_min_i, xlim_max_i, ylim_min_i, ylim_max_i, day, midbrain, duration, bins_speed = np.arange(0, 30, step=0.1))
    DLCp.DLC_histgram_MP_age_speed(genotype,dataframe, save_dir,'mobile', move_th, xlim_min_m, xlim_max_m, ylim_min_m, ylim_max_m, day, midbrain, duration, bins_speed = np.arange(0, 30, step=1))

In [ ]:
dataframe_days = {}

if genotype == 'MitoPark':
    data_ctrl1, data_test1 = DLCp.processed_data(genotype, '', 'SquareOF', '8wks',  'D1', datapath, midbrain, '5_20min')
    data_ctrl2, data_test2 = DLCp.processed_data(genotype, '', 'SquareOF', '16wks', 'D1', datapath, midbrain, '5_20min')
    data_ctrl3, data_test3 = DLCp.processed_data(genotype, '', 'SquareOF', '24wks', 'D1', datapath, midbrain, '5_20min')
    data_ctrl4, data_test4 = DLCp.processed_data(genotype, '', 'Cylinder', '8wks',  'D1', datapath, midbrain, '')
    data_ctrl5, data_test5 = DLCp.processed_data(genotype, '', 'Cylinder', '16wks', 'D1', datapath, midbrain, '')
    data_ctrl6, data_test6 = DLCp.processed_data(genotype, '', 'Cylinder', '24wks', 'D1', datapath, midbrain, '')

    df_combined_sq = pd.concat([data_ctrl1, data_test1,
                                data_ctrl2, data_test2,
                                data_ctrl3, data_test3])
    df_combined_cy = pd.concat([data_ctrl4, data_test4,
                                data_ctrl5, data_test5,
                                data_ctrl6, data_test6])

    dataframe_days[('5_20min', 'D1')] = [df_combined_sq, df_combined_cy]

elif genotype in ['DAT', 'Anxa1']:
    data_ctrl1, data_test1 = DLCp.processed_data(genotype, manipulation, 'SquareOF', '', 'D1', datapath, midbrain, '5_20min')
    data_ctrl2, data_test2 = DLCp.processed_data(genotype, manipulation, 'Cylinder', '', 'D1', datapath, midbrain, '')

    df_combined_sq = pd.concat([data_ctrl1, data_test1])
    df_combined_cy = pd.concat([data_ctrl2, data_test2])

    dataframe_days[('5_20min', 'D1')] = [df_combined_sq, df_combined_cy]
    
df_sq_d1 = dataframe_days[('5_20min', 'D1')][0]
df_cy_d1 = dataframe_days[('5_20min', 'D1')][1]

ax0_y = max(df_sq_d1['immobile_times(s)']) * 1.2
ax1_y = max(df_sq_d1['average_speed_spine2(cm/s)']) * 1.2
ax2_y = max(df_sq_d1['no_of_stops']) * 1.2
ax3_y = max(df_sq_d1['stop_bout_length(s)']) * 1.2
ax4_y = max(df_sq_d1['move_bout_length(s)']) * 1.2
ax5_y = max(df_cy_d1['number_of_rearing']) * 1.2

# ---- Plot with statistics ----
stat = 'Mann-Whitney'
DLCp.DLC_barplots_w_statistics(genotype, manipulation, day, dataframe_days[('5_20min', 'D1')], save_dir, animal, midbrain, duration, stat, ax0_y, ax1_y, ax2_y, ax3_y, ax4_y, ax5_y)